In [7]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

filled_region_mask = Image.open("filled_region_mask.png")

##  # 1. NNField Initialisation

In [8]:
# Tells if a patch is overlapping the target region 
def is_in_bound(y, x, bounds_list, patch_size):
    delta = patch_size // 2
    x_0, y_0 = bounds_list[0], bounds_list[1]
    x_1, y_1 = bounds_list[2], bounds_list[3]
    return (x_0 - delta <= x <= x_1 + delta) and (y_0 - delta <= y <= y_1 + delta)

# Tells if a patch is in the image
def is_in_image(y, x, source_region, patch_size):
    delta = patch_size // 2
    H, W, C = source_region.shape
    return (delta <= y <= H - delta - 1) and (delta <= x <= W - delta - 1)

# Compute source region patches
def compute_valid_patch_centers(filled_region_mask, patch_size):
    delta = patch_size // 2
    H, W = filled_region_mask.shape

    valid_centers = np.ones((H, W), dtype=bool)

    valid_centers[:delta, :] = False
    valid_centers[-delta:, :] = False
    valid_centers[:, :delta] = False
    valid_centers[:, -delta:] = False

    from scipy.ndimage import binary_dilation
    dilated_mask = binary_dilation(filled_region_mask == 0, iterations=delta)
    valid_centers &= ~dilated_mask  

    return valid_centers

    
def initialize_NNField(source_region, target_region, bounds_list, patch_size, valid_centers):
    delta = patch_size // 2
    h, w, c = target_region.shape
    H, W, C = source_region.shape
    NNField = np.zeros((h, w, 3), dtype="int32")

    # Trouver les indices des centres valides
    valid_indices = np.column_stack(np.where(valid_centers))

    for i in range(delta, h - delta):
        for j in range(delta, w - delta):
            # Sélectionner un centre valide aléatoire
            random_idx = np.random.randint(len(valid_indices))
            y_source, x_source = valid_indices[random_idx]
            if valid_centers[y_source,x_source] == False:
                print("patch dans la région cible")
                continue
            NNField[i, j, :2] = [y_source, x_source]

            # Calculer le SSD initial
            target_patch = target_region[i - delta:i + delta + 1, j - delta:j + delta + 1, :]
            source_patch = source_region[y_source - delta:y_source + delta + 1,
                                         x_source - delta:x_source + delta + 1, :]
            diff_all = (source_patch - target_patch) ** 2
            NNField[i, j, 2] = np.sum(diff_all)

    return NNField

##  # 2. Propagation Step

In [9]:
def update_propagation(source_region, target_region, i, j, NNField, patch_size, dx, dy, valid_centers):
    delta = patch_size // 2
    h, w, c = target_region.shape
    H, W, C = source_region.shape
    if not (0 <= i + dy < h and 0 <= j + dx < w):
        return
    offset_neighbor = NNField[i + dy, j + dx, :2]
    y_source_neighbor, x_source_neighbor = offset_neighbor
    target_patch = target_region[i - delta:i + delta + 1, j - delta:j + delta + 1, :]
    source_patch_neighbor = source_region[y_source_neighbor - delta:y_source_neighbor + delta + 1,
                                          x_source_neighbor - delta:x_source_neighbor + delta + 1, :]
    diff_neighbor = (source_patch_neighbor - target_patch) ** 2
    ssd_neighbor = np.sum(diff_neighbor)
    if ssd_neighbor < NNField[i, j, 2]:
        NNField[i, j, :2] = [y_source_neighbor, x_source_neighbor]
        NNField[i, j, 2] = ssd_neighbor

def propagation(source_region,target_region,bounds_list,NNField,patch_size,valid_centers):
    delta = patch_size//2
    h,w,c = target_region.shape
    H,W,C = source_region.shape
    #First step: top to bottom and left to right propagation
    for j in range(1+delta,w-delta):
        for i in range(1+delta,h-delta):
            # top to bottom part
            update_propagation(source_region,target_region,i,j,NNField,patch_size,0,-1,valid_centers)
            # left to right part
            update_propagation(source_region,target_region,i,j,NNField,patch_size,-1,0,valid_centers)
    #Second step: bottom to top and right to left propagation
    for j in range(w-delta-2,delta-1,-1):
        for i in range(h-delta-2,delta-1,-1):
            # bottom to top part
            update_propagation(source_region,target_region,i,j,NNField,patch_size,0,+1,valid_centers)
            # right to left part
            update_propagation(source_region,target_region,i,j,NNField,patch_size,+1,0,valid_centers)

##  # 3. Random search

In [10]:
def random_search(source_region, target_region, NNField, patch_size, valid_centers, n_iter=5):
    delta = patch_size // 2
    h, w, c = target_region.shape
    H, W, _ = source_region.shape

    for i in range(delta, h - delta):
        for j in range(delta, w - delta):
            y_source, x_source = NNField[i, j, :2]
            best_ssd = NNField[i, j, 2]

            for t in range(n_iter):
                search_radius = int(max(H, W) * (0.5 ** t))

                while True:
                    y_random = y_source + np.random.randint(-search_radius, search_radius + 1)
                    x_random = x_source + np.random.randint(-search_radius, search_radius + 1)
                    if (delta <= y_random < H - delta) and (delta <= x_random < W - delta):
                        if valid_centers[y_random, x_random]:
                            break

                source_patch_random = source_region[y_random - delta:y_random + delta + 1,
                                                    x_random - delta:x_random + delta + 1, :]
                target_patch = target_region[i - delta:i + delta + 1, j - delta:j + delta + 1, :]
                ssd = np.sum((source_patch_random - target_patch) ** 2)

                if ssd < best_ssd:
                    NNField[i, j, :2] = [y_random, x_random]
                    NNField[i, j, 2] = ssd
                    best_ssd = ssd

##  # 4. Final reconstruction

In [11]:
# Reconstruction of the target region using patch vote
def construct_target_region(source_region,target_region,bounds_list,NNField,patch_size,valid_centers):
    h,w,c = target_region.shape
    delta = patch_size//2
    result = np.copy(source_region)
    final_target_region = np.zeros_like(target_region,dtype=np.float32)
    patch_map = np.zeros((h,w,3),dtype=np.float32)
    for i in range(delta,h-delta):
        for j in range(delta,w-delta):
            source_y, source_x = NNField[i,j,:2]
            if valid_centers[source_y,source_x] == False:
                print("patch dans la région cible")
            final_target_region[i-delta:i+delta+1,j-delta:j+delta+1] += \
                        source_region[source_y-delta:source_y+delta+1,source_x-delta:source_x+delta+1]
            patch_map[i-delta:i+delta+1,j-delta:j+delta+1,:] += 1
            
    final_target_region /= patch_map
    x_0, y_0 = bounds_list[0],bounds_list[1]
    x_1, y_1 = bounds_list[2],bounds_list[3]
    result[y_0:y_1+1,x_0:x_1+1] = 0
    result[y_0:y_1+1,x_0:x_1+1] = final_target_region
    return result

## # 0. Diffusion Inpainting

In [12]:
def diffusion_inpainting(source_region,filled_region_mask):
    H,W,C = source_region.shape
    filled_region_mask_copy = np.copy(filled_region_mask)
    source_region_copy = np.copy(source_region)
    while np.sum(filled_region_mask_copy == 0) > 0:
        unfilled_points = np.column_stack(np.where(filled_region_mask_copy==0))
        for y,x in unfilled_points:
            sum_around = []
            if x + 1 < W and filled_region_mask_copy[y, x + 1] == 1:
                sum_around.append(source_region_copy[y, x + 1, :])
            if x - 1 >= 0 and filled_region_mask_copy[y, x - 1] == 1:
                sum_around.append(source_region_copy[y, x - 1, :])
            if y + 1 < H and filled_region_mask_copy[y + 1, x] == 1:
                sum_around.append(source_region_copy[y + 1, x, :])
            if y - 1 >= 0 and filled_region_mask_copy[y - 1, x] == 1:
                sum_around.append(source_region_copy[y - 1, x, :])

            if len(sum_around) > 0:
                filled_region_mask_copy[y, x] = 1
                source_region_copy[y, x, :] = np.mean(sum_around, axis=0)
    return source_region_copy

# Test de la diffusion

In [15]:
image = Image.open("nature.png")
filled_region_mask = Image.open("filled_region_mask.png")

# Convertir l'image en tableau NumPy
filled_region_mask = np.array(filled_region_mask)/255
image = np.array(image)/255
print(image.shape)
initial_region_size = np.sum(filled_region_mask == 0)
alpha_channel = image[:,:,3]
image = image[:,:,:3]
occluded_image = np.copy(image)
occluded_image[filled_region_mask==0] = 0 

inpainted_image = diffusion_inpainting(occluded_image,filled_region_mask)
final_image = np.dstack([inpainted_image, alpha_channel])
# Sauvegarder l'image au format PNG

final_image = (255 * final_image).astype(np.uint8)
final_image = Image.fromarray(final_image, 'RGBA')

# Sauvegarder l'image au format PNG
#image.save("image_cercle_color_inpaint2.png")
final_image.save("nature_diffusion.png")

(391, 460, 4)


### Final implementation of patch match

In [14]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage import binary_dilation

image = Image.open("nature.png")
filled_region_mask = Image.open("filled_region_mask.png")

filled_region_mask = np.array(filled_region_mask) / 255
image = np.array(image) / 255
print(image.shape)

alpha_channel = image[:, :, 3]
image = image[:, :, :3]

source_region = np.copy(image)
source_region[filled_region_mask==0] = 0
source_region = diffusion_inpainting(source_region,filled_region_mask)

# Define target region
target_region_shape = np.where(filled_region_mask == 0)
x_min = np.min(target_region_shape[1])
x_max = np.max(target_region_shape[1])
y_min = np.min(target_region_shape[0])
y_max = np.max(target_region_shape[0])
k = 0
bounds_list = [x_min-k, y_min-k,k+x_max,k+y_max]

# Extract target region
target_region = source_region[y_min-k:k+y_max+1, x_min-k:k+x_max+1, :]
print(source_region.shape)
print(target_region.shape)

patch_size = 5

valid_centers = compute_valid_patch_centers(filled_region_mask, patch_size)
print(valid_centers.shape)

# Initialize NNField
NNField = initialize_NNField(source_region, target_region, bounds_list, patch_size, valid_centers)

# Run PatchMatch
n_iter = 3
for i in range(n_iter):
    print(f"Iteration {i}")
    propagation(source_region, target_region, bounds_list, NNField, patch_size, valid_centers)
    random_search(source_region, target_region, NNField, patch_size, valid_centers)
    print("Propagation and random search done")

# Reconstruire l'image finale
final_result = construct_target_region(source_region, target_region, bounds_list, NNField, patch_size,valid_centers)

# Ajouter le canal alpha et sauvegarder l'image
final_image = np.dstack([final_result, alpha_channel])
final_image = (255 * final_image).astype(np.uint8)
final_image = Image.fromarray(final_image, 'RGBA')
final_image.save("nature_patch_match.png")

(391, 460, 4)
(391, 460, 3)
(80, 69, 3)
(391, 460)
Iteration 0
Propagation and random search done
Iteration 1
Propagation and random search done
Iteration 2
Propagation and random search done
